# Classificação de Sentimentos com SVM + Word Embeddings

Este notebook implementa um classificador de sentimentos para avaliações de estabelecimentos utilizando:
- **SVM (Support Vector Machine)**: algoritmo de aprendizado supervisionado para classificação
- **Word Embeddings**: representação vetorial densa de palavras que captura relações semânticas

## Diferença entre BoW e Embeddings

### Bag of Words (BoW)
- Vetores esparsos de alta dimensionalidade
- Cada palavra é independente (não captura relações semânticas)
- Baseado em frequência de palavras

### Word Embeddings
- Vetores densos de menor dimensionalidade (tipicamente 50-300 dimensões)
- Palavras similares têm vetores próximos no espaço vetorial
- Captura relações semânticas e sintáticas
- Exemplos: Word2Vec, GloVe, FastText

## Objetivo
Classificar avaliações de estabelecimentos como positivas ou negativas usando embeddings pré-treinados ou treinados no corpus.

## 1. Importação de Bibliotecas

Importando as bibliotecas necessárias para processamento de texto, embeddings, modelagem e avaliação.

In [1]:
# executar uma única vez
# ! uv run sync
# ! uv run pip install gensim

## troubleshooting with jupyter environment
Solution: Launch Jupyter through the uv environment

  Option 1: Restart Jupyter with uv (Recommended)

  # Stop your current Jupyter server, then run:
  uv run jupyter lab

  This ensures Jupyter uses the Python environment where gensim is installed.

  Option 2: If Jupyter is already running

  1. In your notebook, click on the kernel name in the top-right corner
  2. Select "Python 3 (ipykernel)" from the .venv environment
  3. Restart the kernel

  Option 3: Quick verification in your notebook
  Add this cell to check which Python your notebook is using:

```python
  import sys
  print(sys.executable)
  print(sys.path)
```
  It should show: /Users/jwcunha/Documents/COMPANIES/AB/repos/private/premium/phd/computer-vision-and-nlp/sentiment-analysis/.venv/bin/python

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import re
import string
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados

Carregando o dataset de avaliações do Yelp para análise e treinamento do modelo.

In [3]:
# Carregando o dataset do Yelp
df = pd.read_csv('dataset/yelp_reviews.csv', names=['label', 'text'])

print(f"Dimensões do dataset: {df.shape}")
print(f"\nDistribuição de classes:")
print(df['label'].value_counts())
print(f"\nPrimeiras linhas:")
df.head()

,label,text
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


## 3. Exploração Inicial dos Dados

Verificando a qualidade dos dados e identificando possíveis problemas.

In [4]:
# Verificar valores nulos
print("Valores Nulos:")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

# Informações sobre o dataset
print("Informações do Dataset:")
df.info()
print("\n" + "="*50 + "\n")

# Estatísticas do comprimento dos textos
df['text_length'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

print("Estatísticas de Comprimento:")
print(f"Caracteres - Média: {df['text_length'].mean():.0f}, Mediana: {df['text_length'].median():.0f}")
print(f"Palavras - Média: {df['word_count'].mean():.0f}, Mediana: {df['word_count'].median():.0f}")

Estatísticas de Comprimento:
Caracteres - Média: 723, Mediana: 527
Palavras - Média: 133, Mediana: 97


## 4. Pré-processamento de Texto

Para trabalhar com embeddings, o pré-processamento precisa manter a estrutura das palavras. Vamos:
- Converter para minúsculas
- Remover pontuação
- Remover números
- Tokenizar em palavras (importante para Word2Vec)

In [5]:
def preprocess_text(text):
    """
    Função para pré-processar texto e retornar lista de palavras (tokens)
    """
    # Converter para minúsculas
    text = text.lower()
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remover espaços extras e tokenizar
    tokens = text.split()
    
    return tokens

# Aplicar pré-processamento
df['tokens'] = df['text'].apply(preprocess_text)

# Verificar resultado
print("Exemplo de tokenização:")
print(f"Original: {df['text'].iloc[0][:150]}...")
print(f"\nTokens: {df['tokens'].iloc[0][:20]}...")
print(f"\nNúmero de tokens: {len(df['tokens'].iloc[0])}")

Exemplo de tokenização:
Original: Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and...

Tokens: ['contrary', 'to', 'other', 'reviews', 'i', 'have', 'zero', 'complaints', 'about', 'the', 'service', 'or', 'the', 'prices', 'i', 'have', 'been', 'getting', 'tire', 'service']...

Número de tokens: 124


## 5. Treinamento de Word Embeddings com Word2Vec

Vamos treinar nosso próprio modelo Word2Vec no corpus de avaliações. Os parâmetros principais são:

- **vector_size**: dimensão dos vetores (100 é um bom começo)
- **window**: tamanho da janela de contexto (quantas palavras ao redor considerar)
- **min_count**: frequência mínima de uma palavra para ser incluída no vocabulário
- **workers**: número de threads para processamento paralelo
- **sg**: algoritmo (0 = CBOW, 1 = Skip-gram)

In [6]:
# Treinar modelo Word2Vec
print("Treinando modelo Word2Vec...")
w2v_model = Word2Vec(
    sentences=df['tokens'].tolist(),
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,  # Skip-gram
    epochs=10,
    seed=42
)

print(f"Modelo treinado com sucesso!")
print(f"Tamanho do vocabulário: {len(w2v_model.wv)}")
print(f"Dimensão dos vetores: {w2v_model.wv.vector_size}")

Modelo treinado com sucesso!
Tamanho do vocabulário: 37804
Dimensão dos vetores: 100


### 5.1 Explorando os Embeddings Treinados

Vamos testar a qualidade dos embeddings verificando palavras similares e fazendo operações vetoriais.

In [7]:
# Testar palavras similares
test_words = ['good', 'bad', 'food', 'service', 'excellent', 'terrible']

print("Palavras mais similares:")
print("="*70)

for word in test_words:
    if word in w2v_model.wv:
        similar = w2v_model.wv.most_similar(word, topn=5)
        print(f"\n{word.upper()}:")
        for similar_word, score in similar:
            print(f"  - {similar_word}: {score:.4f}")
    else:
        print(f"\n{word.upper()}: não encontrada no vocabulário")

Palavras mais similares:

GOOD:
  - great: 0.8184
  - decent: 0.8039
  - tasty: 0.7739
  - muchnnthe: 0.7410
  - phenominal: 0.7377

BAD:
  - terrible: 0.7731
  - badbad: 0.7196
  - good: 0.7048
  - horrible: 0.6921
  - poor: 0.6802

FOOD:
  - foodnnthe: 0.7449
  - belowaverage: 0.7196
  - foodit: 0.7126
  - foodnni: 0.7000
  - dinnernnthe: 0.6741

SERVICE:
  - servicennwe: 0.7446
  - servicennthe: 0.7380
  - servicen: 0.7316
  - serviceand: 0.7118
  - servicennoverall: 0.6981

EXCELLENT:
  - outstanding: 0.8419
  - fantastic: 0.8290
  - superb: 0.7952
  - incredible: 0.7915
  - awesome: 0.7853

TERRIBLE:
  - horrible: 0.9302
  - awful: 0.8524
  - lousy: 0.7895
  - bad: 0.7731
  - pitiful: 0.7658


## 6. Criação de Vetores de Documentos

Para usar embeddings de palavras em classificação de documentos, precisamos agregar os vetores das palavras em um único vetor por documento. Vamos usar a **média dos vetores** das palavras do documento.

Estratégias de agregação:
1. **Média**: somar todos os vetores e dividir pelo número de palavras
2. **Média ponderada**: usar TF-IDF como peso
3. **Max pooling**: pegar o máximo de cada dimensão
4. **Doc2Vec**: treinar embeddings diretamente para documentos

In [8]:
def document_vector(tokens, model):
    """
    Calcula o vetor de um documento como a média dos vetores de suas palavras
    """
    # Filtrar palavras que estão no vocabulário
    valid_tokens = [token for token in tokens if token in model.wv]
    
    if len(valid_tokens) == 0:
        # Se nenhuma palavra estiver no vocabulário, retornar vetor zero
        return np.zeros(model.wv.vector_size)
    
    # Calcular média dos vetores
    vectors = [model.wv[token] for token in valid_tokens]
    return np.mean(vectors, axis=0)

# Criar vetores para todos os documentos
print("Criando vetores de documentos...")
doc_vectors = np.array([document_vector(tokens, w2v_model) for tokens in df['tokens']])

print(f"Forma da matriz de features: {doc_vectors.shape}")
print(f"Número de documentos: {doc_vectors.shape[0]}")
print(f"Dimensão dos vetores: {doc_vectors.shape[1]}")
print(f"\nExemplo de vetor (primeiros 10 valores):")
print(doc_vectors[0][:10])

Forma da matriz de features: (38000, 100)
Número de documentos: 38000
Dimensão dos vetores: 100

Exemplo de vetor (primeiros 10 valores):
[ 0.15269439  0.05178944 -0.07896671  0.08913672 -0.09461401  0.03290601
 -0.05525561  0.27215683 -0.19804506  0.07290582]


## 7. Divisão dos Dados em Treino e Teste

Dividindo o dataset em conjuntos de treino (80%) e teste (20%) com estratificação para manter a proporção de classes.

In [9]:
# Preparar features e labels
X = doc_vectors
y = df['label']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")
print(f"Dimensão dos vetores: {X_train.shape[1]}")
print(f"\nDistribuição de classes no treino:")
print(y_train.value_counts())
print(f"\nDistribuição de classes no teste:")
print(y_test.value_counts())

Tamanho do conjunto de treino: 30400 amostras
Tamanho do conjunto de teste: 7600 amostras
Dimensão dos vetores: 100

Distribuição de classes no treino:
label
2    15200
1    15200
Name: count, dtype: int64

Distribuição de classes no teste:
label
1    3800
2    3800
Name: count, dtype: int64


## 8. Treinamento do Modelo SVM

Vamos treinar um SVM com os vetores densos gerados pelos embeddings. Diferente do BoW (esparso), aqui trabalhamos com vetores densos de menor dimensionalidade.

In [10]:
# Treinar modelo SVM inicial
print("Treinando modelo SVM com kernel RBF...")
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")
print(f"\nNúmero de vetores de suporte: {svm_model.n_support_}")
print(f"Vetores de suporte por classe: {dict(zip([1, 2], svm_model.n_support_))}")

Modelo treinado com sucesso!

Número de vetores de suporte: [4640 4647]
Vetores de suporte por classe: {1: np.int32(4640), 2: np.int32(4647)}


### 8.1 Otimização de Hiperparâmetros com GridSearchCV

Vamos otimizar os hiperparâmetros do SVM para embeddings. Como os vetores são densos, kernels não-lineares (RBF) podem funcionar melhor.

In [11]:
# Definir grid de hiperparâmetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Grid Search com validação cruzada
print("Executando Grid Search (pode levar alguns minutos)...")
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("\nMelhores parâmetros encontrados:")
print(grid_search.best_params_)
print(f"\nMelhor score na validação cruzada: {grid_search.best_score_:.4f}")

# Usar o melhor modelo
best_svm = grid_search.best_estimator_


Melhores parâmetros encontrados:
{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}

Melhor score na validação cruzada: 0.9043


## 9. Avaliação do Modelo

Avaliando o desempenho do modelo otimizado no conjunto de teste.

In [12]:
# Fazer predições no conjunto de teste
y_pred = best_svm.predict(X_test)

# Calcular acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("\n" + "="*50 + "\n")

# Relatório de classificação completo
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Negativo (1)', 'Positivo (2)']))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusão:")
print(cm)

Acurácia no conjunto de teste: 0.9067 (90.67%)


Relatório de Classificação:
              precision    recall  f1-score   support

Negativo (1)       0.91      0.91      0.91      3800
Positivo (2)       0.91      0.90      0.91      3800

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600


Matriz de Confusão:
[[3453  347]
 [ 362 3438]]


In [13]:
# Visualização da Matriz de Confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Negativo (1)', 'Positivo (2)'],
            yticklabels=['Negativo (1)', 'Positivo (2)'])
plt.title('Matriz de Confusão - SVM + Embeddings')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predito')
plt.tight_layout()
plt.show()

# Calcular métricas detalhadas
tn, fp, fn, tp = cm.ravel()
print(f"\nVerdadeiros Negativos: {tn}")
print(f"Falsos Positivos: {fp}")
print(f"Falsos Negativos: {fn}")
print(f"Verdadeiros Positivos: {tp}")
print(f"\nTaxa de Falso Positivo: {fp/(fp+tn):.4f}")
print(f"Taxa de Falso Negativo: {fn/(fn+tp):.4f}")


Verdadeiros Negativos: 3453
Falsos Positivos: 347
Falsos Negativos: 362
Verdadeiros Positivos: 3438

Taxa de Falso Positivo: 0.0913
Taxa de Falso Negativo: 0.0953


## 10. Análise de Resultados e Predições

Testando o modelo com exemplos novos e analisando erros.

In [14]:
# Função para predizer sentimento de novos textos
def predict_sentiment(text):
    """
    Prediz o sentimento de um texto usando embeddings
    """
    # Pré-processar e tokenizar
    tokens = preprocess_text(text)
    
    # Criar vetor do documento
    doc_vec = document_vector(tokens, w2v_model).reshape(1, -1)
    
    # Predizer
    prediction = best_svm.predict(doc_vec)[0]
    
    sentiment = "Positivo" if prediction == 2 else "Negativo"
    return sentiment, prediction

# Testar com exemplos novos
test_reviews = [
    "This place is amazing! Best food I've ever had. Highly recommend!",
    "Terrible service, cold food, and overpriced. Never coming back.",
    "It was okay, nothing special but not bad either.",
    "Absolutely loved it! The staff was friendly and the atmosphere was great.",
    "Worst experience ever. Waited for an hour and the food was disgusting."
]

print("Predições para Novos Textos:")
print("="*70)
for i, review in enumerate(test_reviews, 1):
    sentiment, label = predict_sentiment(review)
    print(f"\n{i}. Texto: {review}")
    print(f"   Sentimento: {sentiment} (label={label})")

Predições para Novos Textos:

1. Texto: This place is amazing! Best food I've ever had. Highly recommend!
   Sentimento: Positivo (label=2)

2. Texto: Terrible service, cold food, and overpriced. Never coming back.
   Sentimento: Negativo (label=1)

3. Texto: It was okay, nothing special but not bad either.
   Sentimento: Negativo (label=1)

4. Texto: Absolutely loved it! The staff was friendly and the atmosphere was great.
   Sentimento: Positivo (label=2)

5. Texto: Worst experience ever. Waited for an hour and the food was disgusting.
   Sentimento: Negativo (label=1)


### 10.1 Análise de Erros

Examinar exemplos onde o modelo cometeu erros.

In [15]:
# Identificar predições incorretas
incorrect_predictions = y_test != y_pred
incorrect_indices = y_test[incorrect_predictions].index

print(f"Total de predições incorretas: {incorrect_predictions.sum()}")
print(f"Taxa de erro: {incorrect_predictions.sum() / len(y_test):.4f}\n")

# Mostrar alguns exemplos de erros
print("Exemplos de Classificações Incorretas:")
print("="*70)

num_examples = min(5, len(incorrect_indices))
for i, idx in enumerate(incorrect_indices[:num_examples], 1):
    true_label = y_test.loc[idx]
    pred_label = y_pred[list(y_test.index).index(idx)]
    text = df.loc[idx, 'text']
    
    true_sentiment = "Positivo" if true_label == 2 else "Negativo"
    pred_sentiment = "Positivo" if pred_label == 2 else "Negativo"
    
    print(f"\nExemplo {i}:")
    print(f"Texto: {text[:200]}...")
    print(f"Real: {true_sentiment} ({true_label}) | Predito: {pred_sentiment} ({pred_label})")

Total de predições incorretas: 709
Taxa de erro: 0.0933

Exemplos de Classificações Incorretas:

Exemplo 1:
Texto: Came in late on a Wednesday night about 30 min before closing. This was not my first choice as I was hoping to try some authentic French montreal food. Unfortunately most closed by 10 pm. \n\nIt was l...
Real: Positivo (2) | Predito: Negativo (1)

Exemplo 2:
Texto: We really like the service and food at the bar....
Real: Positivo (2) | Predito: Negativo (1)

Exemplo 3:
Texto: This place is all about the hotel and the shops.  It's a great place to visit and have fun.  They are more expensive than a lot of the other casinos.  The Bourbon Room (bar) is a total rip off.  Don't...
Real: Positivo (2) | Predito: Negativo (1)

Exemplo 4:
Texto: Stayed at a hotel downtown. Went to Diesel on a Saturday night and it was an $8.00 cab ride away. Cover was $5. No coat room ladies, so dress appropriately. Diesel has good music and great VIP table r...
Real: Negativo (1) | Predito: Positi

## 11. Comparação: Embeddings vs Bag of Words

Vamos comparar as principais diferenças entre as duas abordagens.

In [16]:
# Criar tabela comparativa
comparison_data = {
    'Aspecto': [
        'Tipo de Vetor',
        'Dimensionalidade',
        'Esparsidade',
        'Captura Semântica',
        'Contexto',
        'Vocabulário OOV',
        'Tempo de Treinamento',
        'Interpretabilidade'
    ],
    'Bag of Words': [
        'Esparso',
        'Alta (5000+ features)',
        'Muito esparso (>95% zeros)',
        'Não',
        'Ignora ordem e contexto',
        'Vetor zero',
        'Rápido (apenas contagem)',
        'Alta (features = palavras)'
    ],
    'Word Embeddings': [
        'Denso',
        'Baixa (100-300 features)',
        'Denso (sem zeros)',
        'Sim',
        'Captura contexto local',
        'Média de palavras conhecidas',
        'Lento (treinar Word2Vec)',
        'Média (features abstratas)'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("Comparação: SVM + BoW vs SVM + Embeddings")
print("="*80)
print(comparison_df.to_string(index=False))
print("\n" + "="*80)

Comparação: SVM + BoW vs SVM + Embeddings
             Aspecto               Bag of Words              Word Embeddings
       Tipo de Vetor                    Esparso                        Denso
    Dimensionalidade      Alta (5000+ features)     Baixa (100-300 features)
         Esparsidade Muito esparso (>95% zeros)            Denso (sem zeros)
   Captura Semântica                        Não                          Sim
            Contexto    Ignora ordem e contexto       Captura contexto local
     Vocabulário OOV                 Vetor zero Média de palavras conhecidas
Tempo de Treinamento   Rápido (apenas contagem)     Lento (treinar Word2Vec)
  Interpretabilidade Alta (features = palavras)   Média (features abstratas)



## 12. Conclusões

### Resumo do Modelo
- **Algoritmo**: Support Vector Machine (SVM) com kernel otimizado
- **Representação**: Word2Vec embeddings (100 dimensões)
- **Dataset**: Avaliações do Yelp com classificação binária (positivo/negativo)
- **Agregação**: Média dos vetores de palavras por documento

### Vantagens dos Embeddings
1. **Captura semântica**: Palavras similares têm vetores próximos (ex: "good" e "great")
2. **Menor dimensionalidade**: 100 features vs 5000+ do BoW
3. **Vetores densos**: Toda informação é utilizada (sem esparsidade)
4. **Generalização**: Melhor performance em palavras raras devido ao contexto
5. **Transferência de aprendizado**: Pode usar embeddings pré-treinados (GloVe, FastText)

### Desvantagens dos Embeddings
1. **Tempo de treinamento**: Word2Vec precisa ser treinado ou carregado
2. **Complexidade**: Mais difícil de interpretar que BoW
3. **Agregação**: Perda de informação ao calcular média dos vetores
4. **Dependência de corpus**: Embeddings treinados no próprio corpus podem ter vocabulário limitado

### Quando usar cada abordagem?

**Use BoW quando:**
- Velocidade é crítica
- Dataset é grande e vocabulário é rico
- Interpretabilidade é importante
- Recursos computacionais são limitados

**Use Embeddings quando:**
- Qualidade semântica é importante
- Dataset é menor ou tem palavras raras
- Quer aproveitar embeddings pré-treinados
- Pode usar técnicas mais avançadas (CNN, LSTM)

### Próximos Passos
1. Experimentar com embeddings pré-treinados (GloVe, FastText)
2. Testar agregações diferentes (TF-IDF weighted average, max pooling)
3. Usar Doc2Vec para embeddings de documentos
4. Comparar com modelos de deep learning (LSTM, Transformers)
5. Combinar BoW e Embeddings (ensemble)